# VSB,FEI - Generative AI Workshop

The aim of the workshop is to get an overview of data analysis and deep learning techniques in the generative artificial intelligence (GenAI) domain.

* We will use [Python](https://www.python.org/), [Huggingface](https://huggingface.co/) and [Tensorflow](https://www.tensorflow.org/).

**The exercise will cover these topics:**
* GenAI tools for image data using Huggingface models
<!-- * LLM usage for text generating with Huggingface API -->
* Vector representation of text data and searching for similar words using vector distance 
* Design of own deep learning model for generating "Harry Potter"-like text using Keras framework from scratch

## Deep learning in Python introduction
* This lecture is focused on using word embedding for searching for similar words and RNN usage for text generation.

* We will use Harry Potter books in this lectures for demonstration of training own model in Keras and generating our own HP-like stories.

![meme01](https://github.com/rasvob/PopAI-VSB-Workshop/blob/main/images/dl_meme_01.jpg?raw=true)

## Import of the TensorFlow
The main version of the TensorFlow (TF) is a in the Version package in the field VERSION Since the TensformFlow 2.0 everything was encapsulaed under the KERAS api.

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

tf.version.VERSION

'2.13.0'